</center><img src="coverage_workflow_image.png"  width="1000" height="300"/></center>

In [1]:
# import required python libraries
#import geopandas as gpd  # for geo-data structure
import numpy as np  # for some operation better done with numpy
import matplotlib.pyplot as plt  # for creating plots
#import shapely.geometry  # for geometry data crunching
#from shapely.geometry import Point  # specifically for point geometry data manipulation
#import keplergl
#from keplergl import KeplerGl
import datacompy
import pandas as pd
crs = {'init': 'epsg:4326'}
#from shapely.geometry import Point
import mpl_toolkits


In [2]:
# bring in the 3 to 4 different coverage dataset from FB, GSMA, ITU and Ericsson as well as the master school_geolocation

In [3]:
d_f = pd.read_csv('./data/RW_FB_data_20210608.csv',low_memory=False,encoding='utf-8')
d_i = pd.read_csv('./data/coverage_data_itu.csv',low_memory=False,encoding='utf-8')
d_e = pd.read_csv('./data/Rwanda_Ericsson_07_06_2021.csv',low_memory=False,encoding='utf-8')
s_master = pd.read_csv('./data/rwanda_school_geolocation_master.csv',low_memory=False,encoding='utf-8')

In [4]:
# some colunm names standardization

In [5]:
d_e.rename(columns={'SCHOOL_CODE':'school_id', 'SCHOOL_NAME':'school_name'}, inplace=True)
d_i.rename(columns={'giga_schoo': 'giga_school_id','source_sch':'school_id', 'name':'school_name'}, inplace=True)

In [6]:
# cross-profiles the sources with master data for completeness and correctness

In [7]:
# create a separate dfs
master = s_master.copy()
fb = d_f.copy()
ec = d_e.copy()
itu = d_i.copy()
# for facebook
cross_profile_fb_master = datacompy.Compare(
master,
fb,
join_columns='giga_school_id',  
df1_name='Master', 
df2_name='Facebook'
)
#for ericsson
cross_profile_e_master = datacompy.Compare(
master,
ec,
join_columns='giga_school_id',  
df1_name='Master', 
df2_name='Ericsson'
)
# for ITU
cross_profile_itu_master = datacompy.Compare(
master,
itu,
join_columns='giga_school_id',  
df1_name='Master', 
df2_name='ITU'
)

print(cross_profile_fb_master.report())
print(cross_profile_e_master.report())
print(cross_profile_itu_master.report())

DataComPy Comparison
--------------------

DataFrame Summary
-----------------

  DataFrame  Columns  Rows
0    Master       20  4093
1  Facebook       26  4195

Column Summary
--------------

Number of columns in common: 12
Number of columns in Master but not in Facebook: 8
Number of columns in Facebook but not in Master: 14

Row Summary
-----------

Matched on: giga_school_id
Any duplicates on match values: No
Absolute Tolerance: 0
Relative Tolerance: 0
Number of rows in common: 4,091
Number of rows in Master but not in Facebook: 2
Number of rows in Facebook but not in Master: 104

Number of rows with some compared columns unequal: 8
Number of rows with all compared columns equal: 4,083

Column Comparison
-----------------

Number of columns compared with some values unequal: 1
Number of columns compared with all values equal: 11
Total number of values which compare unequal: 8

Columns with Unequal Values or Types
------------------------------------

        Column Master dtype Face

In [8]:
# they usually don't come with "coverage" colunms, we create it at source level using the following codes

In [9]:
d_e['coverage'] = np.where((d_e['2G_coverage']== 'None') & (d_e['3G_coverage']== 'None')& (d_e['4G_coverage']== 'None'), 'NO', 'YES')
d_f['coverage'] = np.where((d_f['percent_2g'].isnull()) & (d_f['percent_3g'].isnull())& (d_f['percent_4g'].isnull()), 'NO', 'YES')
d_f['coverage'] = np.where((d_f['percent_2g']<=0) & (d_f['percent_3g']<=0) & (d_f['percent_4g']<=0), 'NO', 'YES')
d_i['coverage'] = np.where((d_i['2G'].isnull()) & (d_i['3G'].isnull())& (d_i['4G'].isnull()), 'NO', 'YES')

In [10]:
# create the 2G, 3G and 4G coverage columns with binary values at source level using the following codes

In [11]:
d_f['2G_coverage'] = np.where((d_f['percent_2g'] > 0), 'YES', 'NO')
d_f['3G_coverage'] = np.where((d_f['percent_3g'] > 0), 'YES', 'NO')
d_f['4G_coverage'] = np.where((d_f['percent_4g'] > 0), 'YES', 'NO')

In [12]:
d_e.rename(columns={'2G_coverage':'2G_coverage_quality', '3G_coverage':'3G_coverage_quality', '4G_coverage': '4G_coverage_quality'}, inplace=True)

In [13]:
d_e['2G_coverage'] = np.where((d_e['2G_coverage_quality'] != 'None'), 'YES', 'NO')
d_e['3G_coverage'] = np.where((d_e['3G_coverage_quality'] != 'None'), 'YES', 'NO')
d_e['4G_coverage'] = np.where((d_e['4G_coverage_quality'] != 'None'), 'YES', 'NO')
d_i['2G_coverage'] = np.where((d_i['2G'] >=1), 'YES', 'NO')
d_i['3G_coverage'] = np.where((d_i['3G'] == 1), 'YES', 'NO')
d_i['4G_coverage'] = np.where((d_i['4G'] ==1), 'YES', 'NO')

In [14]:
# they usually don't come with "coverage_type" colunms, we create it at source level using the following function and codes

In [15]:
def coverage_type(df):
    if df['4G_coverage'] == 'YES':
        return "4G"
    elif df['3G_coverage'] == 'YES':
        return "3G"
    elif df['2G_coverage'] == 'YES':
        return "2G"
    else:
        return "None"

In [16]:
d_f['coverage_type'] = d_f.apply((lambda x: coverage_type(x)),axis=1)
d_e['coverage_type'] = d_e.apply((lambda x: coverage_type(x)),axis=1)
d_i['coverage_type'] = d_i.apply((lambda x: coverage_type(x)),axis=1)

In [17]:
# isolating the needed colunms from the rest

In [18]:
features = ['giga_school_id','coverage','coverage_type','2G_coverage','3G_coverage','4G_coverage'] 

In [19]:
d_fc = d_f[features]
d_ec = d_e[features]
d_it = d_i[features]

In [20]:
# more sanity checks for nan on giga_school_id

In [21]:
d_fc = d_fc[d_fc['giga_school_id'].notna()]
d_ec = d_ec[d_ec['giga_school_id'].notna()]
d_it = d_it[d_it['giga_school_id'].notna()]

In [22]:
# visually comparing the sources for matches and mismatches across the columns

In [23]:
df_all = pd.concat([d_fc.set_index('giga_school_id'), d_ec.set_index('giga_school_id'),d_it.set_index('giga_school_id')], 
                   axis='columns', keys=['Facebook', 'Ericsson', 'ITU'])
df_all.head()

Facebook                                                    \
               coverage coverage_type 2G_coverage 3G_coverage 4G_coverage   
giga_school_id                                                              
G_RWA_00112         YES            4G         YES         YES         YES   
G_RWA_00116         YES            4G         YES         YES         YES   
G_RWA_00258         YES            4G         YES         YES         YES   
G_RWA_00551         YES            4G         YES         YES         YES   
G_RWA_00901         YES            4G         YES         YES         YES   

               Ericsson                                                    \
               coverage coverage_type 2G_coverage 3G_coverage 4G_coverage   
giga_school_id                                                              
G_RWA_00112         YES            4G         YES         YES         YES   
G_RWA_00116         YES            4G         YES         YES         YES   
G_RWA_00258         YES            4G         YES         YES         YES   
G_RWA_00551         YES            4G         YES         YES         YES   
G_RWA_00901         YES            4G         YES         YES         YES   

                    ITU                                                    
               coverage coverage_type 2G_coverage 3G_coverage 4G_coverage  
giga_school_id                                                             
G_RWA_00112         YES            4G         YES         YES         YES  
G_RWA_00116         YES            4G         YES          NO         YES  
G_RWA_00258         YES            4G         YES          NO         YES  
G_RWA_00551         YES            4G         YES         YES         YES  
G_RWA_00901         YES            4G         YES          NO         YES

In [24]:
df_final = df_all.swaplevel(axis='columns')[d_fc.columns[1:]]
df_final.head()

coverage               coverage_type              2G_coverage  \
               Facebook Ericsson  ITU      Facebook Ericsson ITU    Facebook   
giga_school_id                                                                 
G_RWA_00112         YES      YES  YES            4G       4G  4G         YES   
G_RWA_00116         YES      YES  YES            4G       4G  4G         YES   
G_RWA_00258         YES      YES  YES            4G       4G  4G         YES   
G_RWA_00551         YES      YES  YES            4G       4G  4G         YES   
G_RWA_00901         YES      YES  YES            4G       4G  4G         YES   

                             3G_coverage               4G_coverage           \
               Ericsson  ITU    Facebook Ericsson  ITU    Facebook Ericsson   
giga_school_id                                                                
G_RWA_00112         YES  YES         YES      YES  YES         YES      YES   
G_RWA_00116         YES  YES         YES      YES   NO         YES      YES   
G_RWA_00258         YES  YES         YES      YES   NO         YES      YES   
G_RWA_00551         YES  YES         YES      YES  YES         YES      YES   
G_RWA_00901         YES  YES         YES      YES   NO         YES      YES   

                     
                ITU  
giga_school_id       
G_RWA_00112     YES  
G_RWA_00116     YES  
G_RWA_00258     YES  
G_RWA_00551     YES  
G_RWA_00901     YES

In [25]:
def highlight_diff(data, color='lightcoral'):
    attr = 'background-color: {}'.format(color)
    other = data.xs('Facebook', axis='columns', level=-1)
    return pd.DataFrame(np.where(data.ne(other, level=0), attr, ''),
                        index=data.index, columns=data.columns)

#df_final.style.apply(highlight_diff, axis=None)

In [26]:
# coverage comparison report

In [27]:
# create a separate dfs
fbc = d_fc.copy()
ecc = d_ec.copy()
ituc = d_it.copy()
# Facebook&Ericsson
cross_profile_fb_ec = datacompy.Compare(
fbc,
ecc,
join_columns='giga_school_id',  
df1_name='Facebook', 
df2_name='Ericsson'
)
# Facebook&ITU
cross_profile_fb_itu = datacompy.Compare(
fbc,
ituc,
join_columns='giga_school_id',  
df1_name='Facebook', 
df2_name='ITU'
)
#Ericsson&ITU
cross_profile_Ericsson_itu = datacompy.Compare(
ecc,
ituc,
join_columns='giga_school_id',  
df1_name='Ericsson', 
df2_name='ITU'
)

print(cross_profile_fb_ec.report())
print(cross_profile_fb_itu.report())
print(cross_profile_Ericsson_itu.report())

DataComPy Comparison
--------------------

DataFrame Summary
-----------------

  DataFrame  Columns  Rows
0  Facebook        6  4195
1  Ericsson        6  4197

Column Summary
--------------

Number of columns in common: 6
Number of columns in Facebook but not in Ericsson: 0
Number of columns in Ericsson but not in Facebook: 0

Row Summary
-----------

Matched on: giga_school_id
Any duplicates on match values: No
Absolute Tolerance: 0
Relative Tolerance: 0
Number of rows in common: 4,195
Number of rows in Facebook but not in Ericsson: 0
Number of rows in Ericsson but not in Facebook: 2

Number of rows with some compared columns unequal: 2,504
Number of rows with all compared columns equal: 1,691

Column Comparison
-----------------

Number of columns compared with some values unequal: 5
Number of columns compared with all values equal: 1
Total number of values which compare unequal: 5,948

Columns with Unequal Values or Types
------------------------------------

          Column Face

In [28]:
# merging and selecting the most frequent values across the sources on each row and colunm
# creating a master coverage dataset and merging with the master school location dataset

In [32]:
d_fc.head()

,giga_school_id,coverage,coverage_type,2G_coverage,3G_coverage,4G_coverage
0,G_RWA_00112,YES,4G,YES,YES,YES
1,G_RWA_00116,YES,4G,YES,YES,YES
2,G_RWA_00258,YES,4G,YES,YES,YES
3,G_RWA_00551,YES,4G,YES,YES,YES
4,G_RWA_00901,YES,4G,YES,YES,YES


In [29]:
#Main
df_c = pd.concat([d_fc,d_ec,d_it])
coverage = df_c.groupby(['giga_school_id'],as_index=False).agg(lambda x:x.value_counts().index[0])
master_w_coverage = master.merge(coverage, how="left", on=["giga_school_id"])
master_w_coverage.to_csv('./data/RW_master_w_coverage.csv', encoding='utf-8')
master_w_coverage_connectivity = master_w_coverage.copy()
master_w_coverage_connectivity['connectivity'] = np.where((master_w_coverage_connectivity['connectivity_type'] != 'None'), 'YES', 'NO')
master_w_coverage_connectivity.to_csv('./data/RW_master_w_coverage_connectivity.csv', encoding='utf-8')

In [33]:
master_w_coverage.head(10)

,uuid,country,source,giga_school_id,source_school_id,school_name,education_level,latitude,longitude,hex8,...,internet_availability,number_of_students,number_of_teachers,computers,connectivity_type,coverage,coverage_type,2G_coverage,3G_coverage,4G_coverage
0,cdf4f80c-e3cc-4ebc-8931-8fe76018cc61,Rwanda,government,G_RWA_00002,521312.0,GS MUHORORO,Secondary,-2.159389,29.547972,886ade34e7fffff,...,USB_MODEM,28,1,260.0,3G,YES,4G,YES,NO,YES
1,a58be9dc-59c4-4cf9-96f5-e5b8fa84ba82,Rwanda,government,G_RWA_00005,331220.0,GS MWENDO,Secondary,-1.730639,30.212722,886ad88707fffff,...,USB_MODEM,62,3,272.0,3G,YES,4G,YES,YES,YES
2,9fe38ae7-7d51-4646-841b-1b6daa31c987,Rwanda,government,G_RWA_00006,411305.0,GS NYAGISOZI,Secondary,-2.295028,29.645556,886ade2481fffff,...,USB_MODEM,64,4,218.0,3G,YES,4G,YES,NO,YES
3,8b5c3930-c3ea-44d8-a510-086f8c6889aa,Rwanda,government,G_RWA_00009,510619.0,GS RUGANGO,Secondary,-2.371833,30.599167,886ad821d7fffff,...,USB_MODEM,80,2,252.0,3G,YES,3G,YES,YES,NO
4,2288686a-e13e-423a-b0bb-b7d08452e9a8,Rwanda,government,G_RWA_00012,130610.0,GS GIHINGA,Secondary,-1.926806,30.591806,886ad8a095fffff,...,USB_MODEM,98,3,259.0,3G,YES,4G,YES,YES,YES
5,f74950df-3e95-47d3-8024-d21953768cac,Rwanda,government,G_RWA_00013,421111.0,GS RUYENZI,Secondary,-2.423500,29.827778,886ad84001fffff,...,USB_MODEM,98,3,217.0,3G,YES,4G,YES,YES,YES
6,dee212e0-445a-4981-a8a7-3327105e9aa9,Rwanda,government,G_RWA_00014,520343.0,GS BUREGA,Secondary,-2.159000,29.404111,886ade30bdfffff,...,USB_MODEM,102,3,240.0,3G,YES,4G,YES,NO,YES
7,ae5a9dc6-6feb-446f-9f16-ec1dcf3988d3,Rwanda,government,G_RWA_00020,130140.0,GS KABARE II,Secondary,-2.078583,30.643222,886ad8acabfffff,...,USB_MODEM,155,2,244.0,3G,YES,4G,YES,YES,YES
8,3e6b3ca7-af2c-4768-8280-0db56cff7209,Rwanda,government,G_RWA_00023,521224.0,GS NYAGASIGA,Secondary,-1.710972,30.369194,886ad8b341fffff,...,USB_MODEM,238,15,213.0,3G,YES,2G,YES,YES,NO
9,f016ffce-04f2-45f4-a7f1-ff62ebd87266,Rwanda,government,G_RWA_00032,520335.0,GS MWENDO,Secondary,-2.194306,29.528417,886ade3583fffff,...,USB_MODEM,304,12,394.0,3G,YES,4G,YES,YES,YES


In [41]:
master_w_coverage[master_w_coverage['coverage_type']=='None'].head(10)

,uuid,country,source,giga_school_id,source_school_id,school_name,education_level,latitude,longitude,hex8,...,internet_availability,number_of_students,number_of_teachers,computers,connectivity_type,coverage,coverage_type,2G_coverage,3G_coverage,4G_coverage
37,8b61d879-f3e1-436a-8252-bcbddd083cee,Rwanda,government,G_RWA_00080,550707.0,GS NYAWERA,Secondary,-2.087000,30.889440,886ad9c157fffff,...,FOUR_G,503,16,246.0,4G,NO,None,NO,NO,NO
101,cedfa816-1d36-49b7-af90-75342700696d,Rwanda,government,G_RWA_00279,270703.0,GS KIBINGO,Secondary,-1.753310,29.708530,886adc6d61fffff,...,NONE,1118,23,231.0,None,NO,None,NO,NO,NO
107,9ff7841b-10d6-4525-abf9-555c8e9af038,Rwanda,government,G_RWA_00300,320512.0,GS RWAMIKO,Secondary,-1.939389,29.440028,886adeae3bfffff,...,NONE,1457,32,205.0,None,NO,None,NO,NO,NO
287,628ae4f0-c6ea-498a-b1ed-092f56af7754,Rwanda,government,G_RWA_00795,550608.0,EP KABIRIZI,Primary,-1.759667,29.742111,886adc6d03fffff,...,USB_MODEM,716,11,211.0,3G,NO,None,NO,NO,NO
347,5403ab06-f9f9-4e6d-8738-6a4fd7c7ce68,Rwanda,government,G_RWA_00902,540612.0,GS KIBINGO,Secondary,-1.753306,29.708528,886adc6d61fffff,...,USB_MODEM,773,17,267.0,3G,NO,None,NO,NO,NO
351,6e27a230-2980-4c35-9d7e-b4e2d8e1d0c0,Rwanda,government,G_RWA_00909,510507.0,GS NYAWERA,Secondary,-2.087000,30.889444,886ad9c157fffff,...,USB_MODEM,98,3,202.0,3G,NO,None,NO,NO,NO
376,0e956852-34f9-4246-b7a9-83306d0ef338,Rwanda,government,G_RWA_00966,271006.0,EP KABIRIZI,Primary,-1.759670,29.742110,886adc6d03fffff,...,NONE,337,20,132.0,None,NO,None,NO,NO,NO
838,cf2e4d25-69c8-4fea-b2a3-016ee28db593,Rwanda,government,G_RWA_00267,441003.0,GS RUTOVU,Secondary,-1.346360,29.823080,886adc3905fffff,...,NONE,1009,16,481.0,None,NO,None,NO,NO,NO
983,ce398820-7afd-47ef-b3ec-da07ba8b1f4d,Rwanda,government,G_RWA_00580,361712.0,EP MURAMBO,Primary,-1.362222,29.807056,886adc7653fffff,...,NONE,244,20,141.0,None,NO,None,NO,NO,NO
1036,0ea3a976-2b09-459f-8036-f9fae68a64a1,Rwanda,government,G_RWA_00649,281009.0,EP MUNINI,Primary,-1.740806,29.723028,886adc6d43fffff,...,USB_MODEM,67,4,130.0,3G,NO,None,NO,NO,NO


In [31]:
master_w_coverage_connectivity.head()

,uuid,country,source,giga_school_id,source_school_id,school_name,education_level,latitude,longitude,hex8,...,number_of_students,number_of_teachers,computers,connectivity_type,coverage,coverage_type,2G_coverage,3G_coverage,4G_coverage,connectivity
0,cdf4f80c-e3cc-4ebc-8931-8fe76018cc61,Rwanda,government,G_RWA_00002,521312.0,GS MUHORORO,Secondary,-2.159389,29.547972,886ade34e7fffff,...,28,1,260.0,3G,YES,4G,YES,NO,YES,YES
1,a58be9dc-59c4-4cf9-96f5-e5b8fa84ba82,Rwanda,government,G_RWA_00005,331220.0,GS MWENDO,Secondary,-1.730639,30.212722,886ad88707fffff,...,62,3,272.0,3G,YES,4G,YES,YES,YES,YES
2,9fe38ae7-7d51-4646-841b-1b6daa31c987,Rwanda,government,G_RWA_00006,411305.0,GS NYAGISOZI,Secondary,-2.295028,29.645556,886ade2481fffff,...,64,4,218.0,3G,YES,4G,YES,NO,YES,YES
3,8b5c3930-c3ea-44d8-a510-086f8c6889aa,Rwanda,government,G_RWA_00009,510619.0,GS RUGANGO,Secondary,-2.371833,30.599167,886ad821d7fffff,...,80,2,252.0,3G,YES,3G,YES,YES,NO,YES
4,2288686a-e13e-423a-b0bb-b7d08452e9a8,Rwanda,government,G_RWA_00012,130610.0,GS GIHINGA,Secondary,-1.926806,30.591806,886ad8a095fffff,...,98,3,259.0,3G,YES,4G,YES,YES,YES,YES
